In [0]:
dbutils.fs.rm('dbfs:/FileStore/tables', True)

Out[39]: True

In [0]:
files_name = ['Categories','CustomerCustomerDemo','CustomerDemographics','Customers','Employees','EmployeeTerritories','Orders','Order_Details_','Products','Region','Shippers','Suppliers','Territories']
for file in files_name:
    file_location = "/FileStore/tables/"+file+".csv"
    file_type = 'csv'
    infer_schema = "false"
    first_row_is_header = "true"
    delimiter = ','

    current_table = spark.read.format(file_type)\
    .option("inferSchema", infer_schema)\
    .option("header", first_row_is_header)\
    .option("sep", delimiter)\
    .load(file_location)
    
    current_table.write.parquet("/FileStore/tables/northwind/trusted/"+file+".parquet")

    spark.read.parquet("/FileStore/tables/northwind/trusted/"+file+".parquet").createOrReplaceTempView(file+"_view")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS Date_Dim (
  date_id INT NOT NULL,
  Date DATE,
  Quarter INT,
  Day INT,
  Month INT,
  Year INT
);

INSERT INTO Date_Dim (date_id, Date, Quarter, Day, Month, Year)
SELECT 
  ROW_NUMBER() OVER (ORDER BY OrderDate),
  OrderDate,
  extract(QUARTER FROM OrderDate),
  extract(DAY FROM OrderDate),
  extract(MONTH FROM OrderDate),
  extract(YEAR FROM OrderDate)
FROM Orders_view;

num_affected_rows,num_inserted_rows
830,830


In [0]:
%sql
CREATE TABLE IF NOT EXISTS employee_dim 
(
  EmployeeID INT NOT NULL,
  LastName VARCHAR(20) NOT NULL,
  FirstName VARCHAR(10) NOT NULL,
  Region VARCHAR(15),
  City VARCHAR(15),
  Country VARCHAR(15)
);

INSERT INTO employee_dim (EmployeeID, LastName, FirstName, Region, City, Country)
SELECT 
  EmployeeID,
  LastName,
  FirstName,
  Region,
  City,
  Country
FROM Employees_view;



In [0]:
%sql
SELECT 
  EmployeeID,
  LastName,
  FirstName,
  Region,
  City,
  Country
FROM Employees_view;
--WHERE EmployeeID in (1,2,3,4,5,6,7,8,9);

EmployeeID,LastName,FirstName,Region,City,Country
1,Davolio,Nancy,null,null,null
"Apt. 2A""",Seattle,WA,2,"""Education includes a BA in psychology from Colorado State University in 1970. She also completed """"The Art of the Cold Call."""" Nancy is a member of Toastmasters International.""",null
2,Fuller,Andrew,WA,Tacoma,USA
3,Leverling,Janet,WA,Kirkland,USA
4,Peacock,Margaret,WA,Redmond,USA
5,Buchanan,Steven,null,London,UK
6,Suyama,Michael,null,null,null
"Miner Rd.""",London,null,Portuguese,"""Michael is a graduate of Sussex University (MA, economics, 1983) and the University of California at Los Angeles (MBA, marketing, 1986). He has also taken the courses """"Multi-Cultural Selling"""" and """"Time Management for the Sales Professional."""" He is fluent in Japanese and can read and write French",5
7,King,Robert,null,null,null
"Winchester Way""",London,null,""""" he was transferred to the London office in March 1993.""","""Robert King served in the Peace Corps and traveled extensively before completing his degree in English at the University of Michigan in 1992, the year he joined the company. After completing a course entitled """"Selling in Europe",http://accweb/emmployees/davolio.bmp


In [0]:
%sql
DROP TABLE Date_Dim

In [0]:
# Pegando a tabela em csv
file_location = "/FileStore/tables/Products.csv"
file_type = 'csv'
infer_schema = "false"
first_row_is_header = "true"
delimiter = ','

products = spark.read.format(file_type)\
.option("inferSchema", infer_schema)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load(file_location)

display(products)

ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
1,Chai,1,1,10 boxes x 20 bags,18.0000,39,0,10,0
2,Chang,1,1,24 - 12 oz bottles,19.0000,17,40,25,0
3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.0000,13,70,25,0
4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.0000,53,0,0,0
5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.3500,0,0,0,1
6,Grandma's Boysenberry Spread,3,2,12 - 8 oz jars,25.0000,120,0,25,0
7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30.0000,15,0,10,0
8,Northwoods Cranberry Sauce,3,2,12 - 12 oz jars,40.0000,6,0,0,0
9,Mishi Kobe Niku,4,6,18 - 500 g pkgs.,97.0000,29,0,0,1
10,Ikura,4,8,12 - 200 ml jars,31.0000,31,0,0,0


In [0]:
#Transformando csv em parquet
products.write.parquet("/FileStore/tables/trusted/Products.parquet")

In [0]:
products_parquet = spark.read.parquet("/FileStore/tables/trusted/Products.parquet")
products_parquet.show()

+---------+--------------------+----------+----------+--------------------+---------+------------+------------+------------+------------+
|ProductID|         ProductName|SupplierID|CategoryID|     QuantityPerUnit|UnitPrice|UnitsInStock|UnitsOnOrder|ReorderLevel|Discontinued|
+---------+--------------------+----------+----------+--------------------+---------+------------+------------+------------+------------+
|        1|                Chai|         1|         1|  10 boxes x 20 bags|  18.0000|          39|           0|          10|           0|
|        2|               Chang|         1|         1|  24 - 12 oz bottles|  19.0000|          17|          40|          25|           0|
|        3|       Aniseed Syrup|         1|         2| 12 - 550 ml bottles|  10.0000|          13|          70|          25|           0|
|        4|Chef Anton's Caju...|         2|         2|      48 - 6 oz jars|  22.0000|          53|           0|           0|           0|
|        5|Chef Anton's Gumb...|  

In [0]:
#Transformando em view para usar o sql
products_parquet.createOrReplaceTempView("tabela_products_view")

In [0]:
#Filtrando dados da tabeça de produtos
%sql
SELECT ProductID, ProductName, UnitsOnOrder FROM tabela_products_view

ProductID,ProductName,UnitsOnOrder
1,Chai,0
2,Chang,40
3,Aniseed Syrup,70
4,Chef Anton's Cajun Seasoning,0
5,Chef Anton's Gumbo Mix,0
6,Grandma's Boysenberry Spread,0
7,Uncle Bob's Organic Dried Pears,0
8,Northwoods Cranberry Sauce,0
9,Mishi Kobe Niku,0
10,Ikura,0


In [0]:
#Pegando a tabela de order
file_location = "/FileStore/tables/Order_Details_.csv"
file_type = 'csv'
infer_schema = "false"
first_row_is_header = "true"
delimiter = ','

order_details_csv = spark.read.format(file_type)\
.option("inferSchema", infer_schema)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load(file_location)

display(order_details_csv)

OrderID,ProductID,UnitPrice,Quantity,Discount
10248,11,14.0000,12,0.0
10248,42,9.8000,10,0.0
10248,72,34.8000,5,0.0
10249,14,18.6000,9,0.0
10249,51,42.4000,40,0.0
10250,41,7.7000,10,0.0
10250,51,42.4000,35,0.15
10250,65,16.8000,15,0.15
10251,22,16.8000,6,0.05
10251,57,15.6000,15,0.05


In [0]:
#Transformandp csv em parquet e tranformando em view
order_details_csv.write.parquet("/FileStore/tables/northwind/Order_Details.parquet")

spark.read.parquet("/FileStore/tables/northwind/Order_Details.parquet").createOrReplaceTempView("order_view")

In [0]:

%sql
/*Pegando produtos por quantidade de vendas */
SELECT ProductID, count(OrderID) AS total_order FROM order_view
GROUP BY ProductID
ORDER BY total_order asc

ProductID,total_order
9,5
37,6
48,6
15,6
66,8
27,9
5,10
67,10
50,10
3,12


In [0]:
%sql
/* pegando os 3 menos vendidos com os nomes */
SELECT count(OrderID) AS total_orders, vo.ProductID, pv.ProductName
FROM order_view AS vo
INNER JOIN tabela_products_view AS pv
ON vo.ProductID = pv.ProductID
GROUP BY vo.ProductID, pv.ProductName
ORDER BY total_orders asc
LIMIT 3;

total_orders,ProductID,ProductName
5,9,Mishi Kobe Niku
6,15,Genen Shouyu
6,48,Chocolade


In [0]:
%sql
/* pegando os 10 mais vendidos com os nomes */
SELECT count(OrderID) AS total_orders, vo.ProductID, pv.ProductName
FROM order_view AS vo
INNER JOIN tabela_products_view AS pv
ON vo.ProductID = pv.ProductID
GROUP BY vo.ProductID, pv.ProductName
ORDER BY total_orders desc
LIMIT 10;

total_orders,ProductID,ProductName
54,59,Raclette Courdavault
51,60,Camembert Pierrot
51,24,Guaran� Fant�stica
51,31,Gorgonzola Telino
50,56,Gnocchi di nonna Alice
48,62,Tarte au sucre
47,41,Jack's New England Clam Chowder
46,75,Rh�nbr�u Klosterbier
44,2,Chang
43,16,Pavlova


In [0]:
file_location = "/FileStore/tables/Customers.csv"
file_type = "csv"
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

customer_csv = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)

display(customer_csv)

CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,null,12209,Germany,030-0074321,030-0076545
ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constituci�n 2222,M�xico D.F.,null,05021,Mexico,(5) 555-4729,(5) 555-3745
ANTON,Antonio Moreno Taquer�a,Antonio Moreno,Owner,Mataderos 2312,M�xico D.F.,null,05023,Mexico,(5) 555-3932,null
AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,null,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
BERGS,Berglunds snabbk�p,Christina Berglund,Order Administrator,Berguvsv�gen 8,Lule�,null,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
BLAUS,Blauer See Delikatessen,Hanna Moos,Sales Representative,Forsterstr. 57,Mannheim,null,68306,Germany,0621-08460,0621-08924
BLONP,Blondesddsl p�re et fils,Fr�d�rique Citeaux,Marketing Manager,"24, place Kl�ber",Strasbourg,null,67000,France,88.60.15.31,88.60.15.32
BOLID,B�lido Comidas preparadas,Mart�n Sommer,Owner,"C/ Araquil, 67",Madrid,null,28023,Spain,(91) 555 22 82,(91) 555 91 99
BONAP,Bon app',Laurence Lebihan,Owner,"12, rue des Bouchers",Marseille,null,13008,France,91.24.45.40,91.24.45.41
BOTTM,Bottom-Dollar Markets,Elizabeth Lincoln,Accounting Manager,23 Tsawassen Blvd.,Tsawassen,BC,T2F 8M4,Canada,(604) 555-4729,(604) 555-3745


In [0]:
file_location = "/FileStore/tables/Orders.csv"
file_type = "csv"
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

orders_csv = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)

display(orders_csv)

OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
10248,VINET,5,1996-07-04 00:00:00.000,1996-08-01 00:00:00.000,1996-07-16 00:00:00.000,3,32.3800,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,null,51100,France
10249,TOMSP,6,1996-07-05 00:00:00.000,1996-08-16 00:00:00.000,1996-07-10 00:00:00.000,1,11.6100,Toms Spezialit�ten,Luisenstr. 48,M�nster,null,44087,Germany
10250,HANAR,4,1996-07-08 00:00:00.000,1996-08-05 00:00:00.000,1996-07-12 00:00:00.000,2,65.8300,Hanari Carnes,"Rua do Pa�o, 67",Rio de Janeiro,RJ,05454-876,Brazil
10251,VICTE,3,1996-07-08 00:00:00.000,1996-08-05 00:00:00.000,1996-07-15 00:00:00.000,1,41.3400,Victuailles en stock,"2, rue du Commerce",Lyon,null,69004,France
10252,SUPRD,4,1996-07-09 00:00:00.000,1996-08-06 00:00:00.000,1996-07-11 00:00:00.000,2,51.3000,Supr�mes d�lices,"Boulevard Tirou, 255",Charleroi,null,B-6000,Belgium
10253,HANAR,3,1996-07-10 00:00:00.000,1996-07-24 00:00:00.000,1996-07-16 00:00:00.000,2,58.1700,Hanari Carnes,"Rua do Pa�o, 67",Rio de Janeiro,RJ,05454-876,Brazil
10254,CHOPS,5,1996-07-11 00:00:00.000,1996-08-08 00:00:00.000,1996-07-23 00:00:00.000,2,22.9800,Chop-suey Chinese,Hauptstr. 31,Bern,null,3012,Switzerland
10255,RICSU,9,1996-07-12 00:00:00.000,1996-08-09 00:00:00.000,1996-07-15 00:00:00.000,3,148.3300,Richter Supermarkt,Starenweg 5,Gen�ve,null,1204,Switzerland
10256,WELLI,3,1996-07-15 00:00:00.000,1996-08-12 00:00:00.000,1996-07-17 00:00:00.000,2,13.9700,Wellington Importadora,"Rua do Mercado, 12",Resende,SP,08737-363,Brazil
10257,HILAA,4,1996-07-16 00:00:00.000,1996-08-13 00:00:00.000,1996-07-22 00:00:00.000,3,81.9100,HILARION-Abastos,Carrera 22 con Ave. Carlos Soublette #8-35,San Crist�bal,T�chira,5022,Venezuela


In [0]:
# Tranformar para parquet

# customer_csv.write.format("parquet").option("header", "true").save("dbfs:/FileStore/tables/northwind/Customer_Details")
orders_csv.write.format("parquet").option("header", "true").save("dbfs:/FileStore/tables/northwind/Orders")

In [0]:
orders_csv.printSchema()

root
 |-- OrderID: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- EmployeeID: string (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- RequiredDate: string (nullable = true)
 |-- ShippedDate: string (nullable = true)
 |-- ShipVia: string (nullable = true)
 |-- Freight: string (nullable = true)
 |-- ShipName: string (nullable = true)
 |-- ShipAddress: string (nullable = true)
 |-- ShipCity: string (nullable = true)
 |-- ShipRegion: string (nullable = true)
 |-- ShipPostalCode: string (nullable = true)
 |-- ShipCountry: string (nullable = true)



In [0]:
# Lendo parquet e transformando em view

spark.read.parquet("dbfs:/FileStore/tables/northwind/Customer_Details").createOrReplaceTempView("customer_view")
spark.read.parquet("dbfs:/FileStore/tables/northwind/Orders").createOrReplaceTempView("order_view")

In [0]:
%sql
/* pegando os 10 mais vendidos com os nomes */
SELECT cv.CustomerID, count(ov.OrderID) AS total_orders 
FROM customer_view AS cv
INNER JOIN order_view AS ov
ON cv.CustomerID = ov.CustomerID
GROUP BY cv.CustomerID
ORDER BY total_orders desc
LIMIT 5;

CustomerID,total_orders
SAVEA,31
ERNSH,30
QUICK,28
FOLKO,19
HUNGO,19
BERGS,18
RATTC,18
HILAA,18
BONAP,17
WARTH,15
